In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, json
from scipy.sparse import csr_matrix

### Load in the challenge dataset

In [2]:
os.system("unzip dataset.zip")

256

In [3]:
# Read in the JSON data
data_stream = ""
with open("challenge_set.json", "r") as f:
    data_stream = f.read()
data_set = json.loads(data_stream)

In [4]:
# Grab unique track/artist pairs
track_uris = set()
for i,j in enumerate(data_set["playlists"]):
    if len(j["tracks"]) == 100:
        for track in j["tracks"]:
            track_uris.add((track["track_name"], track["artist_name"], track["track_uri"]))
unique_tracks = list(track_uris)

In [5]:
uri_to_title_artist_map = {x[2]: (x[0], x[1]) for x in unique_tracks}
for i,(j,k) in enumerate(uri_to_title_artist_map.items()):
    if i > 20: break
    print(j,k)

spotify:track:0nnxf8p851ZR49KFnGWVsB ('Say Hey (I Love You)', 'Michael Franti & Spearhead')
spotify:track:3Q9YjVy5ql7BDL4OoEAE78 ('Lay Low', 'Blake Shelton')
spotify:track:0qt5f5EL92o8Snzopsv0en ('Geronimo', 'Sheppard')
spotify:track:5ArstqTuXWHd9If9OlvDSn ('Get This Party Started', 'Stretch Arm Strong')
spotify:track:4gpmL5xSZeHssgyIP3UuPS ('What Do You Do', 'A$AP Ferg')
spotify:track:6aG75URCRp44sjzp5lUf4P ('One Little Kiss', 'Dallas Smith')
spotify:track:5faaMYhs6QHvMEtvMrvuAe ('What Would You Do', 'Bastille')
spotify:track:45wBTYlOx3FsuFluuuRRQh ('Love Me Harder', 'Ariana Grande')
spotify:track:37wvYILIEROhoaOR4CqG5m ('Outside - Aaron Lewis and Fred Durst', 'Staind')
spotify:track:2S2od3hT7ceytw7d1pTRuE ('Again', 'Noah Cyrus')
spotify:track:0NyOeqzqjVM9j7IuWp2oJ6 ('Feel Good (feat. Daya)', 'Gryffin')
spotify:track:6MIwdFXHnImyz1TmsuUrEB ('Bound To You', 'Jocelyn Alice')
spotify:track:2AtI9RfFqGxskGfBdyPWbg ('All Of This', 'blink-182')
spotify:track:36xxq7VDZZsGwARsn6Clo7 ('Living S

### Load in the audio feature data scraped from the spotify API

In [6]:
df = pd.read_csv("all_downloaded_data.csv")
df = df.drop("Unnamed: 0", axis=1)
df = df.set_index('uri')
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms,time_signature
uri,,,,,,,,,,,,,,,,,
spotify:track:38uqMByifM0mkjcTbejlyR,0.375,0.294,8,-8.751,1,0.0329,0.928000,0.000000,0.1900,0.125,95.128,audio_features,38uqMByifM0mkjcTbejlyR,https://api.spotify.com/v1/tracks/38uqMByifM0m...,https://api.spotify.com/v1/audio-analysis/38uq...,161573,4
spotify:track:4kOTDj4SiZPkW1p96WMDNv,0.950,0.803,1,-4.195,1,0.1510,0.145000,0.000000,0.0748,0.860,117.000,audio_features,4kOTDj4SiZPkW1p96WMDNv,https://api.spotify.com/v1/tracks/4kOTDj4SiZPk...,https://api.spotify.com/v1/audio-analysis/4kOT...,213066,4
spotify:track:1c9LBfX0klXJscJAYDI0Ea,0.758,0.501,0,-7.051,1,0.0589,0.252000,0.000000,0.1910,0.397,78.042,audio_features,1c9LBfX0klXJscJAYDI0Ea,https://api.spotify.com/v1/tracks/1c9LBfX0klXJ...,https://api.spotify.com/v1/audio-analysis/1c9L...,194147,4
spotify:track:59iNA6gwthoTJ6IgpBGGHW,0.534,0.749,6,-5.562,0,0.0749,0.077600,0.000001,0.0906,0.671,179.975,audio_features,59iNA6gwthoTJ6IgpBGGHW,https://api.spotify.com/v1/tracks/59iNA6gwthoT...,https://api.spotify.com/v1/audio-analysis/59iN...,212160,4
spotify:track:4QCng4MDyUhEZ730Ma2Rjp,0.674,0.744,11,-5.196,1,0.2740,0.023800,0.000000,0.3300,0.624,154.086,audio_features,4QCng4MDyUhEZ730Ma2Rjp,https://api.spotify.com/v1/tracks/4QCng4MDyUhE...,https://api.spotify.com/v1/audio-analysis/4QCn...,262400,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
spotify:track:3SI9uxzQCSEfj7V1Qvf7nU,0.394,0.641,1,-7.645,1,0.3730,0.247000,0.000004,0.6560,0.193,165.120,audio_features,3SI9uxzQCSEfj7V1Qvf7nU,https://api.spotify.com/v1/tracks/3SI9uxzQCSEf...,https://api.spotify.com/v1/audio-analysis/3SI9...,389800,4
spotify:track:5DrRvw69AtJ0PwVpEp22NO,0.810,0.562,1,-4.593,1,0.1300,0.050700,0.000000,0.1220,0.179,119.892,audio_features,5DrRvw69AtJ0PwVpEp22NO,https://api.spotify.com/v1/tracks/5DrRvw69AtJ0...,https://api.spotify.com/v1/audio-analysis/5DrR...,241091,4
spotify:track:4zYBVaPx3EfZIUoNWFtdov,0.563,0.697,8,-6.127,0,0.0439,0.000575,0.000000,0.2240,0.653,148.040,audio_features,4zYBVaPx3EfZIUoNWFtdov,https://api.spotify.com/v1/tracks/4zYBVaPx3EfZ...,https://api.spotify.com/v1/audio-analysis/4zYB...,247408,4


### Create a matrix mapping each song to membership in a playlist

In [7]:
playlist_idcs = []
for i,j in enumerate(data_set["playlists"]):
    if len(j["tracks"]) == 100:
        playlist_idcs.append(i) 
print(len(unique_tracks))
print(len(playlist_idcs), "total playlists")
print(df.shape[0], "total songs")

53552
2000 total playlists
53550 total songs


### Build playlist membership matrix from scratch, this might take a while, i've compressed the matrix and unzip it below to save time.
```python

membership_mtx = np.zeros((len(unique_tracks), len(playlist_idcs)), dtype="u1")

raw = [data_set["playlists"][pdx] for pdx in playlist_idcs]
playlists = []
for data in raw:
    playlists.append([d["track_uri"] for d in data["tracks"]])

for row in range(0, len(unique_tracks)):
    r_uri = unique_tracks[row][2]
    for idx, playlist in enumerate(playlists):
        if r_uri in playlist:
            membership_mtx[row, idx] = 1     
    if row % 1000 == 0: print(row)
        
np.save("playlist_membership_labels.npy", unique_tracks)
np.save("playlist_membership_data.npy", membership_mtx)
```

In [8]:
os.system("unzip playlist_membership_data.zip")

256

In [9]:
membership_mtx = np.load("playlist_membership_data.npy")
membership_lbls = np.load("playlist_membership_labels.npy")

In [10]:
print("Avg song is in", np.mean(np.sum(membership_mtx, axis=1)), "playlists")

Avg song is in 3.695492231849417 playlists


### Create matrix of counts two songs appear in the same playlist

In [11]:
# This is a super sparse matrix, so use csr format to save space.
test = csr_matrix(membership_mtx)
frequency_csr_mtx = test @ test.T

In [15]:
# Identify the two songs with the highest pairwise count.
dense_frequency_mtx = frequency_csr_mtx.todense() #Creating a dense array is probably not a good idea (crashes sometimes lol)
np.fill_diagonal(dense_frequency_mtx, 0)

max_elem = np.unravel_index(dense_frequency_mtx.argmax(), dense_frequency_mtx.shape)
print("dense_mtx indices =", max_elem)
dense_frequency_mtx[max_elem]
print()
print("The most commonly associated songs are:")
print("\t", membership_lbls[max_elem[0]][0], "by", membership_lbls[max_elem[0]][1])
print("\t", membership_lbls[max_elem[1]][0], "by", membership_lbls[max_elem[1]][1])
print(f"Appeared in the same playlist {dense_frequency_mtx[max_elem]}/2000 times")

dense_mtx indices = (5948, 20514)

The most commonly associated songs are:
	 Broccoli (feat. Lil Yachty) by DRAM
	 Caroline by Aminé
Appeared in the same playlist 57/2000 times
